[View in Colaboratory](https://colab.research.google.com/github/tyronedamasceno/brazilian-feelings/blob/master/brazilian_feelings.ipynb)

# Análise de sentimentos por região

Uma das áreas de ciência de dados que tem apresentado grande crescimento é a análise de sentimentos.

Nesse notebook vamos ver uma forma de analisar como está o sentimento dos usuários do twitter sobre ~~politica~~ um determinado assunto. Além disso essa análise poderá ser feitas sobre as regiões, visto que um mesmo assunto pode causar diferentes reações em locais variados.

## Pré-requisitos

- Python3
- TextBlob
- TweePy
- Numpy
- Conta no twitter


In [2]:
!pip install tweepy textblob numpy
!python -m textblob.download_corpora

    100% |████████████████████████████████| 634kB 21.7MB/s 
    100% |████████████████████████████████| 286kB 28.7MB/s 
  Running setup.py bdist_wheel for PySocks ... - done
  Stored in directory: /root/.cache/pip/wheels/22/5c/b5/12e0dfdfa85bea67b23628b6425fae715c687e947a45ee3df9
Successfully built PySocks
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/mo

## Perfil de desenvolvedor

Após instalar as bibliotecas será necessário (se você não possuir ainda) criar uma conta de desenvolvedor no twitter.

Isso pode ser feito acessando a página de [desenvolvedores](https://developer.twitter.com) do twitter (logado na sua conta) e clicar em _apply for a development account_

Após sua conta ser aceita (as vezes é rápido, as vezes demora...) você precisa criar um app e guardar as credenciais dele.

- Consumer API keys
- Access token & access token secret

## Vamos a luta!

Agora de posse de tudo que é necessário, mãos a obra!

In [0]:
import tweepy
import numpy as np
from textblob import TextBlob

In [0]:
consumer_key='W313gbqFgBk7bJrXYvxbFJCeu'
consumer_secret='pGXNhrHKeymEGpmXr3Tgng7TdQJ8DQ4ZvC1X0GsielppPACbZE'

access_token='1050013375940517888-rehrUfWWzUc3LXeHYsXolu8q2UoxaS'
access_token_secret='Q8t7SLe3olzODJ800GkbkHef8NMnXTpaT6H6a7s2wrAh2'

Agora, pra confirmar se deu tudo certo, façamos a autenticação na API do twitter

In [5]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)

api

Se deu tudo certo, vamos ao que interessa, buscar os dados para trabalhar!!!

In [0]:
tweets = api.search('Bolsonaro -filter:retweets')

O trecho "-filter:retweets" serve para que os retweets não sejam trazidos para nossa base, evitando que haja repetição de dados

Agora, vamos iterar sobre todos os tweets recuperados, colocando o conteúdo de texto (tweet.text) em um TextBlob

In [0]:
for tweet in tweets:
  print(TextBlob(tweet.text))

Um "problema" que temos é que o algoritmo de análise de sentimentos do TextBlob foi treinado em inglês, portanto usaremos um recurso que faz a tradução para inglês dos tweets que tiverem em outros idiomas. 

Primeiro criaremos uma função que detecta se o tweet está em inglês

In [0]:
def is_english(text):
  if text.detect_language() == "en":
    return True
  return False

Dentro do laço for vamos verificar se o texto está em inglês, se não estiver faremos a tradução deste. Em seguida, faremos a análise de sentimentos

In [0]:
for tweet in tweets:
  text = TextBlob(tweet.text)
  
  if not is_english(text):
    text = TextBlob(str(text.translate(to='en')))
  
  #print('Tweet: ' + tweet.text)
  #print('Polarity: ' + str(text.sentiment.polarity) + " \ " + str(text.sentiment.subjectivity))
  #print('.....................')

## Agora, um pouco de teoria

Do nada um monte de dados que não sabemos de onde vem, né? Calma que vamos explicar tudo...

**Polarity**:  Um valor entre -1.0 e 1.0, onde -1.0 se refere a uma polaridade 100% negativa, 1.0 uma polaridade 100% positiva e o 0 se refere a neutralidade

**Subjectivity**: Um valor entre 0.0 e 1.0, onde 0.0 se refere a um valor 100% objetivo e 1.0 se refere a um valor 100%subjetivo

Sentenças objetivas normalmente possuem fatos ou informaçãoes, enquanto sentenças subjetivas expressam sentimentos pessoais e opiniões

## Voltando ao trabalho

Agora que sabemos o que os dados significam, devemos ignorar valores com onde sua polaridade é neutra e são 100% objetivas, visto que se referem a fatos, e fatos não expressam sentimentos.

Vamos juntar tudo que vimos dentro de uma função para ficar mais organizado

In [0]:
def tweet_analysis():
    polarities = []

    for tweet in tweets:
        text = TextBlob(tweet.text)

        if not is_english(text):
            text = TextBlob(str(text.translate(to='en')))
            
        if (text.sentiment.polarity != 0.0 and text.sentiment.subjectivity != 0.0):
            polarities.append(text.sentiment.polarity)

        #print('Tweet: ' + tweet.text)
        #print('Polarity: ' + str(text.sentiment.polarity) + " \ " + str(text.sentiment.subjectivity))
        #print('.....................')
        
    
    return polarities

E usaremos o **NumPy** para calcular a média das polaridades

In [12]:
polarity_mean = np.mean(tweet_analysis())

print('Média: ' + str(polarity_mean))
if(polarity_mean > 0.0):
    print('POSITIVE')
else:
    print('NEGATIVE')

Média: 0.06493055555555555
POSITIVE


## Êba, pronto??? Ainda não

De fato criamos um analisador de tweets, mas esse projeto vai muito além disso.

Vamos alterar a forma de busca dos tweets para que possamos determinar o número de tweets recuperados e que esses sejam os mais recentes

In [0]:
tweets = tweepy.Cursor(api.search, q="Python Brasil -filter:retweets", result_type="recent").items(20)

## Modificando um pouco as coisas

Agora, faremos uma mudança na nossa função `tweet_analysis()` para que essa retorne um dicionário com as polaridades e subjetividades de cada tweet, bem como ela que receba o termo a ser passado como query e o numero de tweets a serem procurados.

In [0]:
def tweet_analysis(query, items=20):
    
    tweets = tweepy.Cursor(api.search, wait_on_rate_limit=True, q=query + " -filter:retweets", result_type="recent").items(items)
    polarities = []
    subjectivities = []
    
    for tweet in tweets:
        text = TextBlob(tweet.text)
        #if not is_english(text):
        #    text = TextBlob(str(text.translate(to='en')))
            
        
        if (text.sentiment.polarity != 0.0 and text.sentiment.subjectivity != 0.0):
            polarities.append(text.sentiment.polarity)
            subjectivities.append(text.sentiment.subjectivity)
        
    
    return {'polarity': polarities, 'subjectivity':subjectivities}

Vamos também criar uma função que pega a média ponderada dos tweets a partir da sua subjetividade e outra para imprimir os resultados

In [0]:
def get_weighted_polarity_mean(valid_tweets):
    return np.average(valid_tweets['polarity'],weights=valid_tweets['subjectivity'])

def get_polarity_mean(valid_tweets):
    return np.mean(valid_tweets['polarity'])
  
def print_result(mean):
    if mean > 0.0:
        print('POSITIVE')
    elif mean == 0.0:
        print('NEUTRO')
    else:
        print('NEGATIVE')

## Finalizando a análise

Vamos criar uma função principal que centraliza a análise:



In [0]:
def do_analysis():
  query = input("Entre a query de analise: ")
  analysis = tweet_analysis(query)

  print('MÉDIA PONDERADA: ' + str(get_weighted_polarity_mean(analysis)))
  print_result(get_weighted_polarity_mean(analysis))

  print('MÉDIA: ' + str(get_polarity_mean(analysis)))
  print_result(get_polarity_mean(analysis))

In [94]:
do_analysis()

Entre a query de analise: Trump
MÉDIA PONDERADA: 0.15503915599211138
POSITIVE
MÉDIA: 0.08037918871252207
POSITIVE
